In [3]:
import numpy as np
from src.utils_py.gro.Structure import Structure
from src.utils_py.io.gro import write_gro, read_gro

---

# Файл нужен для того, чтобы сделать щель с двумя отзеркаленными подложками

In [24]:
# file = '2x1_final.gro'
# file = 'cal_substrate_10x6x2.gro'
file = 'cal_substrate_5x12x2.gro'

In [25]:
structure = read_gro(f'cal_104/{file}')

In [6]:
structure.atoms.shape[0]

38400

In [26]:
new_strucute = Structure(
    box = structure.box,
    atoms = np.zeros(2 * structure.atoms.shape[0], dtype=object),
    atoms_xyz= np.zeros((2 * structure.atoms_xyz.shape[0], 3))
)

In [27]:
z_1 = 0.089 # z of first Ca layer
z_2 = 0.396 # z of second Ca layer
dz_offset = z_2 - 3 * z_1

In [28]:
id = 1
# Добавление основной подложки
for i, atom_label in np.ndenumerate(structure.atoms):
    new_atom_label = atom_label.copy()
    new_atom_label.id = id
    new_strucute.add_atom(new_atom_label, structure.atoms_xyz[i[0], :].copy(), id-1)
    id += 1

# Добавление зеркальной подложки
for i, atom_label in np.ndenumerate(structure.atoms):
    new_atom_label = atom_label.copy()
    new_atom_label.id = id

    new_xyz = structure.atoms_xyz[i[0], :].copy()
    new_xyz[2] *= -1
    new_xyz[2] -= dz_offset
    new_strucute.add_atom(new_atom_label, new_xyz, id-1)
    id += 1

In [29]:
z_min = np.min(new_strucute.atoms_xyz[:, 2])

In [30]:
# Поднятие подложки
for i, atom_label in np.ndenumerate(new_strucute.atoms):
    new_strucute.atoms_xyz[i, 2] -= z_min

In [31]:
z_max = np.max(new_strucute.atoms_xyz[:, 2])

In [32]:
new_strucute.box[2] = z_max

In [33]:
with open(f'cal_104/cal_5x12x2.gro', 'w') as f:
    f.write(write_gro(new_strucute))